In [ ]:
# Prerequisites
import json
import os
from eye_ai.eye_ai import EyeAI
from deriva_ml.dataset_bag import DatasetBag

import pandas as pd
from pathlib import Path, PurePath
import logging

from deriva_ml.deriva_ml_base import MLVocab as vc
from deriva_ml.execution_configuration import ExecutionConfiguration, Workflow, Execution
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

In [ ]:
# Login
from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
# host = 'www.eye-ai.org'
host = 'dev.eye-ai.org'
catalog_id = "eye-ai"

gnl = GlobusNativeLogin(host=host)
if gnl.is_logged_in([host]):
    print("You are already logged in.")
else:
    gnl.login([host], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")

In [ ]:
cache_dir = '/data'
working_dir = '/data'
EA = EyeAI(hostname = host, catalog_id = catalog_id, cache_dir= cache_dir, working_dir=working_dir)

# Configuration

In [ ]:
# Add Workflow Vocab terms
EA.add_term(vc.workflow_type, "Test Workflow", description="A test Workflow for new DM")
# Workflow instance
test_workflow = Workflow(
    name="Test New Workflow_LAC",
    url="https://github.com/informatics-isi-edu/eye-ai-exec/blob/main/notebooks/templates/template_lac.ipynb",
    workflow_type="Test Workflow"
)
# Configuration instance
config = ExecutionConfiguration(
    bdbags=['2-7K8W'],
    models = ['2-4JR6'],
    execution=Execution(description="Sample Test Execution"),
    workflow=test_workflow,
    description="Our Test Workflow instance")
# Initialize execution
configuration_record = EA.initialize_execution(config)
execution_rid = configuration_record.execution_rid

In [ ]:
print(configuration_record)

# Create DatasetBag

In [ ]:
ds_bag = DatasetBag(configuration_record.bag_paths[0])

# Create Feature

In [ ]:
new_feature = EA.create_feature(feature_name = "The Name of the feature", 
                                table = "The table name that feature attached to", 
                                terms = ["Table name of the cv columns"], 
                                comment="Any comments")

feature_image_angle = EA.create_feature(feature_name = "Fundus_Angle", table = "Image", 
                                        terms = ["Image_Angle_Vocab"], comment="Angle of fundus images")
feature_image_laterality = EA.create_feature(feature_name = "Fundus_Laterality", table = "Image", 
                                             terms = ["Image_Side_Vocab"], comment="Lateraliity of fundus images")

# Retrive a feature

In [ ]:
# Find all the features that attached to a table
EA.find_features(table_name)

In [ ]:
Feature = EA.feature_record_class(table_name, feature_name)

In [ ]:
# display all the columns in a feature table
display(
    Markdown('### Feature Name'),
    [ f'Name: {c.name} Required {not c.nullok}' for c in DiagnosisFeature.feature.feature_columns]
)

# Ingest new values to a feature table

In [ ]:
# Format ingestion values
diag_feature_list = [Feature(
    Execution=configuration_record.execution_rid,
    Image= image_rid,
    col1 = ,
    col2 = ,
    Feature_Name = feature_name,
    ) for image_rid in image_rid_list]

# E.g. Diagnosis 
diag_feature_list = [DiagnosisFeature(
    Execution=configuration_record.execution_rid,
    Image= image_rid,
    Diagnosis_Image= ["No Glaucoma", "Suspected Glaucoma", "Unknown"][random.randint(0,2)],
    Image_Quality= ["Good", "Bad", "Unknown"][random.randint(0,2)],
    Feature_Name = "Diagnosis",
    Cup_Disk_Ratio = 0.9000,
    Diagnosis_Tag = "test_feature") for image_rid in ['2-BDAM', '2-BDAP', '2-BDAR']]

In [ ]:
# Ingest new values to the feature table
EA.add_features(diag_feature_list)

# Upload results

In [ ]:
# crete asset path
asset_type_name = ""
asset_path = configuration_record.execution_assets_path(asset_type_name)
# save assets to asset_path


# upload assets to catalog
EA.upload_execution(configuration_record, True)